In [1]:
import re
import csv
from collections import defaultdict, Counter


In [2]:
log_file = "sample.log"

with open(log_file, 'r') as file:
    logs = file.readlines()


In [3]:
ip_counts = Counter()

for log in logs:
    match = re.search(r'(\d+\.\d+\.\d+\.\d+)', log)
    if match:
        ip_counts[match.group(1)] += 1

# Sort IP addresses by request count in descending order
sorted_ip_counts = sorted(ip_counts.items(), key=lambda x: x[1], reverse=True)

print("IP Address           Request Count")
for ip, count in sorted_ip_counts:
    print(f"{ip:20} {count}")


IP Address           Request Count
203.0.113.5          8
198.51.100.23        8
192.168.1.1          7
10.0.0.2             6
192.168.1.100        5


In [4]:
endpoint_counts = Counter()

for log in logs:
    match = re.search(r'\"(?:GET|POST|PUT|DELETE|HEAD) (/\S*)', log)
    if match:
        endpoint_counts[match.group(1)] += 1

# Find the most accessed endpoint
most_accessed = endpoint_counts.most_common(1)[0]

print("\nMost Frequently Accessed Endpoint:")
print(f"{most_accessed[0]} (Accessed {most_accessed[1]} times)")



Most Frequently Accessed Endpoint:
/login (Accessed 13 times)


In [5]:
failed_login_counts = Counter()
threshold = 10  # Configurable threshold for failed login attempts

for log in logs:
    if "401" in log or "Invalid credentials" in log:
        match = re.search(r'(\d+\.\d+\.\d+\.\d+)', log)
        if match:
            failed_login_counts[match.group(1)] += 1

# Filter IPs exceeding the threshold
suspicious_ips = {ip: count for ip, count in failed_login_counts.items() if count > threshold}

print("\nSuspicious Activity Detected:")
print("IP Address           Failed Login Attempts")
for ip, count in suspicious_ips.items():
    print(f"{ip:20} {count}")



Suspicious Activity Detected:
IP Address           Failed Login Attempts


In [6]:
output_file = "log_analysis_results.csv"

with open(output_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write IP Requests
    csv_writer.writerow(["IP Address", "Request Count"])
    csv_writer.writerows(sorted_ip_counts)
    csv_writer.writerow([])  # Empty row for separation

    # Write Most Accessed Endpoint
    csv_writer.writerow(["Most Accessed Endpoint", "Access Count"])
    csv_writer.writerow([most_accessed[0], most_accessed[1]])
    csv_writer.writerow([])  # Empty row for separation

    # Write Suspicious Activity
    csv_writer.writerow(["IP Address", "Failed Login Count"])
    csv_writer.writerows(suspicious_ips.items())

print("\nResults saved to", output_file)



Results saved to log_analysis_results.csv
